# Covid-19 Detector 

In [1]:
# For deeplearning
import tensorflow as tf       
from tensorflow import keras 
from keras.utils.np_utils import to_categorical 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
# For managing files   
import os
# For image manipulation
from PIL import Image
import cv2
import imutils
# Vectorized code, translate images into vector arrays
import numpy as np
# Data processing, confusion matrix and classification report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder 
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import classification_report
# For visualization and image display
import matplotlib.pyplot as plt
import cv2
import imutils
from google.colab.patches import cv2_imshow
# Utilities
from collections import Counter
import copy
import random

: 

: 

In [ ]:
def dict_labels(path):
    '''Returns a dictionary with the name of the picture and the appropriate labels'''
    with open(path) as f:
        lines = f.readlines()
    split_lines = [line.split() for line in lines]

    return {split_line[1]:split_line[2] for split_line in split_lines}

def load_colorize_images(path, path_doc, image_size=(112,112)): 
    '''Returns two arrays: one with the pseudo-colorized images and another one with the output labels'''

    data = []
    titles = []

    for file in os.listdir(path):
        path_file = path + '/' + file
        img = cv2.imread(path_file,0)
        img = cv2.resize(img, image_size)
        img = cv2.applyColorMap(img, cv2.COLORMAP_BONE)
        img = (img/255).astype(np.float32)
        data.append(img)
        titles.append(file) 

    dict_data_labels = dict_labels(path_doc)

    return np.array(data), np.array([dict_data_labels[title] 
                                     for title in titles])
